# SETUP

In [153]:
import numpy as np
import itertools
import requests
import datetime
import time
import json
from datetime import datetime
import pickle
import pandas as pd
import rasterio as rio

# Requests 

From GEO EDA sheet:

## DarkSky Sample

1) cheapest seems to be DarkSky. 1000 per day for free 2) $0.0001 per request after that 3) Request: " https://api.darksky.net/forecast/[key]/40.036944,-121.005833,2005-02-02T17:30:00" (my api key removed)

Output is like this: {"latitude":40.036944,"longitude":-121.005833,"timezone":"America/Los_Angeles","currently":{"time":1107394200,"precipIntensity":0,"precipProbability":0,"temperature":50.24,"apparentTemperature":50.41,"dewPoint":13.85,"humidity":0.23,"windSpeed":2.62,"windGust":13.17,"windBearing":24,"uvIndex":0},"hourly":{"data":
...
{"time":1107414000,"precipIntensity":0,"precipProbability":0,"temperature":41.2,"apparentTemperature":41.2,"dewPoint":12.53,"humidity":0.31,"windSpeed":1.56,"windGust":12.79,"windBearing":100,"uvIndex":0}]},"daily":{"data":[{"time":1107331200,"sunriseTime":1107357120,"sunsetTime":1107393960,"moonPhase":0.78,"precipIntensity":0,"precipIntensityMax":0,"precipProbability":0,"temperatureHigh":57.95,"temperatureHighTime":1107384600,"temperatureLow":38.42,"temperatureLowTime":1107439200,"apparentTemperatureHigh":57.45,"apparentTemperatureHighTime":1107384600,"apparentTemperatureLow":38.91,"apparentTemperatureLowTime":1107439200,"dewPoint":14.85,"humidity":0.31,"windSpeed":3.33,"windGust":14.83,"windGustTime":1107390840,"windBearing":76,"uvIndex":0,"uvIndexTime":1107331200,"temperatureMin":35.03,"temperatureMinTime":1107359100,"temperatureMax":57.95,"temperatureMaxTime":1107384600,"apparentTemperatureMin":34.3,"apparentTemperatureMinTime":1107338640,"apparentTemperatureMax":57.45,"apparentTemperatureMaxTime":1107384600}]},"flags":{"sources":["cmc","gfs","hrrr","icon","isd","madis","nam","sref"],"nearest-station":3.312,"units":"us"},"offset":-8}

## Dark Sky locality

In [52]:
# From Geo Sheet - Toy Data Bounding Box
# keep in mind each pixel is roughly 500 m x 500 m
bb_long = [-122, -119.912,-119.912,-122,-122 ]
bb_lat = [36.8, 36.8, 35.06, 35.06,36.8]

Requests, to see how how 'hyperlocal' the darksky data really is.

In [53]:
# Create Pairs
min_long = min(bb_long)
max_long = max(bb_long)
min_lat = min(bb_lat)
max_lat = max(bb_lat)

In [54]:
spaces = 10
longs = np.linspace(min_long,max_long,spaces)
lats = np.linspace(min_lat,max_lat,spaces)
print(longs, lats)
distance = (max_lat-min_lat)*110/spaces
print(f"distance between forecast pairings ~ {distance:.3} km, or {(distance/.5):.3} pixels")

[-122.    -121.768 -121.536 -121.304 -121.072 -120.84  -120.608 -120.376
 -120.144 -119.912] [35.06       35.25333333 35.44666667 35.64       35.83333333 36.02666667
 36.22       36.41333333 36.60666667 36.8       ]
distance between forecast pairings ~ 19.1 km, or 38.3 pixels


In [55]:
# Pairs
pairs = [(lat,long) for lat in lats for long in longs]
print(len(pairs))
print(pairs[25])

100
(35.446666666666665, -120.84)


In [56]:
key = '5ffac5f056d341c6296cba58fa96e9ba'

The Forecast Data API supports HTTP compression. We heartily recommend using it, as it will make responses much smaller over the wire. To enable it, simply add an `Accept-Encoding: gzip` header to your request.

### Try one request

In [57]:
# items to exclude from call
blocks = '[currently,minutely,hourly,alerts]'
# Units for call
units = 'ca'
# Time
time = '2005-02-02T12:00:00' # for noon, but hour doesn't matter as we're grabbing daily data only.
# Dates (relevant to our fires): only use one for testing
dates = ['2016-07-21']

# create time string:
date = dates[0]
time = date+'T12:00:00'
lat = str(pairs[25][0])
long = str(pairs[25][1])

query = ('https://api.darksky.net/forecast/'+key+'/'+ 
        lat+','+long+','+time+'?exclude=' 
        +blocks+'&units='+units)
headers = {'Accept-Encoding':'gzip'}
print(query)   

https://api.darksky.net/forecast/5ffac5f056d341c6296cba58fa96e9ba/35.446666666666665,-120.84,2016-07-21T12:00:00?exclude=[currently,minutely,hourly,alerts]&units=ca


In [58]:
r=requests.get(query,headers=headers)

In [59]:
r.status_code

200

In [61]:
weather = r.json()
weather['daily']['data']

[{'time': 1469084400,
  'summary': 'Foggy in the morning.',
  'icon': 'clear-day',
  'sunriseTime': 1469106300,
  'sunsetTime': 1469157360,
  'moonPhase': 0.58,
  'precipIntensity': 0,
  'precipIntensityMax': 0,
  'precipProbability': 0,
  'temperatureHigh': 32.15,
  'temperatureHighTime': 1469138220,
  'temperatureLow': 13.71,
  'temperatureLowTime': 1469194800,
  'apparentTemperatureHigh': 31.87,
  'apparentTemperatureHighTime': 1469138220,
  'apparentTemperatureLow': 13.98,
  'apparentTemperatureLowTime': 1469194800,
  'dewPoint': 9.95,
  'humidity': 0.56,
  'pressure': 1016.3,
  'windSpeed': 6.1,
  'windGust': 20.82,
  'windGustTime': 1469145600,
  'windBearing': 268,
  'cloudCover': 0.08,
  'uvIndex': 11,
  'uvIndexTime': 1469131740,
  'visibility': 15.083,
  'temperatureMin': 11.36,
  'temperatureMinTime': 1469104980,
  'temperatureMax': 32.15,
  'temperatureMaxTime': 1469138220,
  'apparentTemperatureMin': 11.63,
  'apparentTemperatureMinTime': 1469104980,
  'apparentTemperature

In [62]:
# Parsed important elements:
dailydata = weather['daily']['data'][0]
rain = dailydata['precipIntensityMax']
hitemp = dailydata['temperatureHigh']
lotemp = dailydata['temperatureLow']
humidity = dailydata['humidity']
windspd = dailydata['windSpeed']
winddir = dailydata['windBearing']
clouds = dailydata['cloudCover']

In [63]:
point = [date,lat,long,rain,hitemp,lotemp,humidity,windspd,winddir,clouds]

In [64]:
point

['2016-07-21',
 '35.446666666666665',
 '-120.84',
 0,
 32.15,
 13.71,
 0.56,
 6.1,
 268,
 0.08]

In [65]:
print(type(point))

<class 'list'>


In [66]:
pairs[0][0]

35.06

## Multiple Requests to determine degree of hyperlocality required

In [99]:
def getweather(coordinates,date,key):
    """ Function to take the set of coordinates and get the daily
    weather for a given date. rain, hi and low temps, humidity
    wind speed and direction and cloud cover only.
    
    Input: Date, API Key and list of coordinate pairs (lat,long)
    Output: list of jsons
    """
    # items to exclude from call
    blocks = '[currently,minutely,hourly,alerts]'

    # Units for call
    units = 'ca'
    
    # Compression for call
    headers = {'Accept-Encoding':'gzip'}
    
    time = date+'T12:00:00'
    
    data_out = []
    for pair in coordinates:
        lat = str(pair[0])
        long = str(pair[1])
        # set the query string for darksky
        query = ('https://api.darksky.net/forecast/'+key+'/'+ 
            lat+','+long+','+time+'?exclude=' 
            +blocks+'&units='+units)
        # Make the call to Dark Sky to get all the data for that date and location
        r=requests.get(query,headers=headers)
        
        # get the weather data from the request return
        weather=r.json()
        
        # write the jsons as items in the output data list
        data_out.append(weather)
    
    return data_out

In [100]:
%%time
data_for_date = getweather(pairs,date,key)

Wall time: 33 s


In [68]:
# %%time
# # Pickle results out so that i don't have to make another api call later
# with open('../data/GlobalFire2016/weathertest.pickle','wb') as f:
#     pickle.dump(data_out,f,pickle.HIGHEST_PROTOCOL)

Wall time: 6 ms


In [80]:
def check_point(info):
    """ to check if there is a value for that weather element and provide 
    either None or the value back
    """
    if info:
        return info
    else:
        return None

In [104]:
def date_df(data_out):
    """ Function to take the list of jsons from a date
    of daily weather requests and create a pandas dataframe
    
    Input: list of jsons containing the weather data
    Output: pandas dataframe
    """
    data_convert=[]
    # Titles for the data
    coltitles = ['date','latitude','longitude','rain','High T','Low T','Humidity','Wind Speed','Wind Direction','Cloud Cover']
    data_convert.append(coltitles)
    # Loope through all the geopoints to get the data
    for point in data_out:
        lat = point['latitude']
        long = point['longitude']
        daily = point.get('daily')
        if daily:
            data = daily['data'][0]
            time = datetime.fromtimestamp(data['time']).strftime('%Y-%m-%d')
            rain = data.get('precipIntensityMax')
            hitemp = data.get('temperatureHigh')
            lotemp = data.get('temperatureLow')
            humidity = data.get('humidity')
            windspd = data.get('windSpeed')
            winddir = data.get('windBearing')
            clouds = data.get('cloudCover')
            point = [date,lat,long,check_point(rain), \
                     check_point(hitemp),check_point(lotemp), \
                     check_point(humidity),check_point(windspd), \
                     check_point(winddir), check_point(clouds)] 
            data_convert.append(point)
        else:
            point = None
            
    df = pd.DataFrame(data_convert[1:], columns = data_convert[0])
    
    return df


In [105]:
df2 = date_df(data_for_date)
df2

,date,latitude,longitude,rain,High T,Low T,Humidity,Wind Speed,Wind Direction,Cloud Cover
0,2016-07-21,35.06,-121.536,None,15.09,13.82,NaN,33.49,327.0,NaN
1,2016-07-21,35.06,-121.304,None,15.09,13.82,0.84,33.49,327.0,NaN
2,2016-07-21,35.06,-121.072,None,20.81,13.73,0.77,7.59,275.0,0.20
3,2016-07-21,35.06,-120.840,None,22.47,13.97,0.71,6.77,274.0,0.37
4,2016-07-21,35.06,-120.608,None,22.16,13.00,0.73,6.60,247.0,0.38
...,...,...,...,...,...,...,...,...,...,...
90,2016-07-21,36.80,-120.840,None,33.55,14.14,0.37,10.03,336.0,NaN
91,2016-07-21,36.80,-120.608,None,35.21,15.74,0.34,10.91,326.0,NaN
92,2016-07-21,36.80,-120.376,None,35.17,15.74,0.33,10.00,321.0,NaN
93,2016-07-21,36.80,-120.144,None,36.14,16.75,0.34,7.93,308.0,0.01


In [127]:
stats = df2.describe()

In [142]:
cols = stats.columns

In [ ]:
uniques = []
for col in cols:
    uniq = len(df2[col].value_counts())
    uniques.append(uniq)
uniques = pd.DataFrame([uniques],columns=cols)

In [ ]:
uniques.rename(index={0:'Uniques'},inplace = True)

In [151]:
newstats = stats.append(uniques)
newstats

,latitude,longitude,High T,Low T,Humidity,Wind Speed,Wind Direction,Cloud Cover
count,95.000000,95.000000,95.000000,95.000000,90.000000,95.000000,94.000000,45.000000
mean,35.969684,-120.910821,29.696211,13.639895,0.491333,10.315368,279.042553,0.179111
std,0.543820,0.655386,6.749324,3.248108,0.166458,8.650780,88.161498,0.117818
min,35.060000,-122.000000,15.090000,6.050000,0.280000,4.510000,4.000000,0.010000
25%,35.446667,-121.536000,25.425000,11.275000,0.340000,6.605000,277.250000,0.080000
50%,36.026667,-120.840000,32.040000,13.330000,0.450000,7.440000,306.000000,0.200000
75%,36.413333,-120.376000,35.235000,15.740000,0.657500,9.260000,322.000000,0.280000
max,36.800000,-119.912000,38.660000,20.560000,0.840000,42.090000,358.000000,0.390000
Uniques,10.000000,10.000000,88.000000,88.000000,42.000000,89.000000,62.000000,27.000000


This says that the data is fairly hyperlocal. Try doing heat maps?
To do heatmaps, have to turn this into a numpy array which i have to do anyway

In [162]:
# Create Numpy Array for High T first:
X,Y = np.meshgrid(longs,lats)

In [174]:
# for each latitude, create a list of all the values by increasing longitude, so we can vstack into array
#for lat in lats:
sub = df2[['latitude','longitude','High T']]
sub
row = sub[sub['latitude']==35.06].sort_values(by=['longitude'])
print(row)
print(longs)
    

   latitude  longitude  High T
0     35.06   -121.536   15.09
1     35.06   -121.304   15.09
2     35.06   -121.072   20.81
3     35.06   -120.840   22.47
4     35.06   -120.608   22.16
5     35.06   -120.376   26.98
6     35.06   -120.144   32.68
7     35.06   -119.912   34.33
[-122.    -121.768 -121.536 -121.304 -121.072 -120.84  -120.608 -120.376
 -120.144 -119.912]


In [158]:
with rio.open('../data/GlobalFire2016/Global_fire_atlas_firelinecrop.tif','r') as dst:
    meta = dst.meta
    fireline = dst.read(1)

In [159]:
meta

{'driver': 'GTiff',
 'dtype': 'int32',
 'nodata': -9999.0,
 'width': 456,
 'height': 470,
 'count': 1,
 'crs': CRS.from_epsg(3857),
 'transform': Affine(509.9863732168427, 0.0, -13581080.848881941,
        0.0, -509.9928633270912, 4411268.819459678)}

In [160]:
fireline

array([[-9999, -9999, -9999, ..., -9999, -9999, -9999],
       [-9999, -9999, -9999, ..., -9999, -9999, -9999],
       [-9999, -9999, -9999, ..., -9999, -9999, -9999],
       ...,
       [-9999, -9999, -9999, ..., -9999, -9999, -9999],
       [-9999, -9999, -9999, ..., -9999, -9999, -9999],
       [-9999, -9999, -9999, ..., -9999, -9999, -9999]])